In [2]:

from sklearn.manifold import TSNE
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizerFast
from transformers import BertTokenizerFast, BertForSequenceClassification



/projects/c7a1100d-10f9-4de1-b172-d38d7d09565f/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:

from transformers import BertTokenizerFast, BertForSequenceClassification

# Define the directory where you saved the model and tokenizer
output_dir = './model_save/'

# Load the tokenizer
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Load the model
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states=True)



In [4]:
df = pd.read_csv("embed_data")
df = df.loc[(df.label == "standard") | (df.label == "requirement")]
print(df.groupby("document").count())
print(df.head())

                text  label
document                   
AASHTO            27     27
ASABE             39     39
ATSM              99     99
IEEE              13     13
ISO               17     17
JCanadaWelding   350    350
Jpierburg        214    214
Jtoho            159    159
MilSpec            9      9
SKADish          289    289
SKAMid           291    291
                                                text        label  \
0  CSP_Mid.CBF shall have a Maintenance Down Time...  requirement   
1  When commanded, CSP_Mid.CBF shall perform auto...  requirement   
2   Each box end end blast station paddle lift sh...  requirement   
3   Each pipe shall be transferred into the box e...  requirement   
4   Each of the vrollers shall be used on many st...  requirement   

         document  
0          SKAMid  
1          SKAMid  
2  JCanadaWelding  
3  JCanadaWelding  
4  JCanadaWelding  


In [5]:
requirements = df.text.tolist()
labels = df.label.tolist()
documents = df.document.tolist()
#print(documents)

In [6]:
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Tokenize and encode the text data
encoded_text = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in requirements]

# Obtain BERT embeddings for each text
embeddings = [model(**text).last_hidden_state.mean(dim=1).squeeze().detach().numpy() for text in encoded_text]


In [15]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(np.array(embeddings))


# Create a DataFrame with the t-SNE embeddings and labels
df = pd.DataFrame(embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
df['label'] = labels
df['document_class'] = df['document']  # Create a new column for document classes

# Create a dictionary to map document classes to unique colors
unique_document_classes = df['document_class'].unique()
colors = sns.color_palette('husl', n_colors=len(unique_document_classes))
document_class_color_mapping = {doc_class: color for doc_class, color in zip(unique_document_classes, colors)}

# Assign a color to each data point based on its document class
df['color'] = df['document_class'].map(document_class_color_mapping)

# Create a scatter plot of the t-SNE embeddings for different document classes
plt.figure(figsize=(10, 8))
sns.set(style='whitegrid')  # Set Seaborn style with gridlines

for doc_class, color in document_class_color_mapping.items():
    df_class = df[df['document_class'] == doc_class]
    marker = 's' if doc_class == 'Triangle' else 'o'
    sns.scatterplot(data=df_class, x='Dimension 1', y='Dimension 2', hue='document_class', palette=[color], marker=marker, edgecolor='k', s=100)
    


# Create legends for the document classes
legend_handles = [plt.Line2D([0], [0], marker=marker, color='w', label=doc_class, markersize=10,
                              markerfacecolor=color) for doc_class, color in document_class_color_mapping.items()]
plt.legend(handles=legend_handles, title='Document Classes')

plt.title("t-SNE Visualization of Embeddings with Document Classes and IDs", fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)  # Turn off gridlines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the plot as a PNG image
plt.savefig('tsne_visualization_with_classes.png', dpi=300, bbox_inches='tight')

plt.show()



KeyError: 'document'

In [6]:
def requirement_to_embedding(model, tokenizer, requirement):
    input = tokenizer(requirement, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        input = input.to("cpu")  # Copy input to CPU
        outputs = model(**input)  # Run model without labels to get outputs
        embeddings = outputs.hidden_state
        embeddings = embeddings.detach().numpy()
    return embeddings

In [7]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
embeddings = [ requirement_to_embedding ( model , tokenizer , requirement ) for
requirement in requirements ]
embedd_array = np . stack ( embeddings )
embedd_array . shape


AttributeError: 'SequenceClassifierOutput' object has no attribute 'hidden_state'

In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from transformers import BertTokenizer, BertModel
import torch

# Load a pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Tokenize and encode the text data
encoded_requirements = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128, padding='max_length', return_tensors='pt') for text in requirements]

# Generate BERT embeddings
bert_embeddings = [bert_model(tokens)['last_hidden_state'].mean(dim=1).squeeze().detach().numpy() for tokens in encoded_requirements]

# Initialize the t-SNE model with specified perplexity and learning rate
perplexity_value = 50
learning_rate_value = 10
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

# Fit the t-SNE model to the BERT embeddings
embeddings_2d = tsne.fit_transform(bert_embeddings)

# Convert labels to binary values (0 for 'requirement', 1 for 'standard')
binary_labels = [0 if label == 'requirement' else 1 for label in labels]

# Define colors and markers for the two classes
colors = sns.color_palette('Set1', n_colors=2)
markers = 'o'

# Create a DataFrame with the t-SNE embeddings and binary labels
df_tsne = pd.DataFrame(embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
df_tsne['label'] = binary_labels

# Create a scatter plot of the t-SNE embeddings for both classes
plt.figure(figsize=(10, 8))
sns.set(style='whitegrid')

for label in set(binary_labels):
    df_label = df_tsne[df_tsne['label'] == label]
    sns.scatterplot(data=df_label, x='Dimension 1', y='Dimension 2', hue='label', palette=[colors[label]], markers=markers, edgecolor='k', s=100)

# Create legends for the classes
class_labels = ['requirement', 'standard']
legend_handles = [plt.Line2D([0], [0], marker=markers, color='w', label=class_labels[label], markersize=10, markerfacecolor=colors[label]) for label in set(binary_labels)]
plt.legend(handles=legend_handles, title='Classes')

plt.title("t-SNE Visualization of BERT Embeddings with Class Labels", fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the plot as a PNG image
plt.savefig('tsne_visualization.png', dpi=300, bbox_inches='tight')

plt.show()



We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


AttributeError: 'list' object has no attribute 'shape'

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Assuming embedd_array is a numpy array of embeddings
# And document is a list of labels corresponding to each embedding

# Create an empty dictionary to store embeddings for each class
class_embeddings = {}

# Group embeddings by class
for i, label in enumerate(documents):
    if label not in class_embeddings:
        class_embeddings[label] = []
    class_embeddings[label].append(normalized_embeddings[i])

# Calculate the average cosine distance for each class
class_avg_distances = {}

for label, embeddings in class_embeddings.items():
    # Calculate cosine distances within each class
    distances = 1 - cosine_similarity(embeddings)
    # Calculate the average distance for this class
    avg_distance = np.mean(distances)
    class_avg_distances[label] = avg_distance

# Now, class_avg_distances contains the average cosine distance for each class
for label, avg_distance in class_avg_distances.items():
    print(f"Class {label}: Average Cosine Distance = {avg_distance}")


The following cell was generated by ChatGPT using the prompt:

> is there a visual way to show the output of the above cell?

 

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

# Assuming embedd_array is a numpy array of embeddings
# And documents is a list of labels corresponding to each embedding

# Create an empty dictionary to store embeddings for each class
class_embeddings = {}

# Group embeddings by class
for i, label in enumerate(documents):
    if label not in class_embeddings:
        class_embeddings[label] = []
    class_embeddings[label].append(normalized_embeddings[i])

# Calculate inter-class cosine distances
inter_class_distances = {}

# Iterate through unique pairs of classes
unique_classes = list(class_embeddings.keys())
for i in range(len(unique_classes)):
    for j in range(i + 1, len(unique_classes)):
        class1 = unique_classes[i]
        class2 = unique_classes[j]
        
        # Calculate cosine similarity between embeddings of class1 and class2
        distances = 1 - cosine_similarity(class_embeddings[class1], class_embeddings[class2])
        
        # Calculate the average distance for this pair of classes
        avg_distance = np.mean(distances)
        
        # Store the average distance in the dictionary
        inter_class_distances[(class1, class2)] = avg_distance

# Convert inter_class_distances into a distance matrix
n_classes = len(unique_classes)
distance_matrix = np.zeros((n_classes, n_classes))

for i, class1 in enumerate(unique_classes):
    for j, class2 in enumerate(unique_classes):
        if i < j:
            distance_matrix[i, j] = inter_class_distances[(class1, class2)]
        elif i > j:
            distance_matrix[i, j] = inter_class_distances[(class2, class1)]

# Convert the distance_matrix to its condensed form
condensed_distance_matrix = pdist(distance_matrix)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_distance_matrix, method='average')

# Create a dendrogram
plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, labels=unique_classes, orientation='top', leaf_font_size=10)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Classes")
plt.ylabel("Distance")
plt.show()


In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming embedd_array is a numpy array of embeddings
# And documents is a list of labels corresponding to each embedding

# Create an empty dictionary to store embeddings for each class
class_embeddings = {}

# Group embeddings by class
for i, label in enumerate(documents):
    if label not in class_embeddings:
        class_embeddings[label] = []
    class_embeddings[label].append(normalized_embeddings[i])

# Calculate inter-class cosine distances
inter_class_distances = {}

# Iterate through unique pairs of classes
unique_classes = list(class_embeddings.keys())
for i in range(len(unique_classes)):
    for j in range(i + 1, len(unique_classes)):
        class1 = unique_classes[i]
        class2 = unique_classes[j]
        
        # Calculate cosine similarity between embeddings of class1 and class2
        distances = 1 -cosine_similarity(class_embeddings[class1], class_embeddings[class2])
        
        # Calculate the average distance for this pair of classes
        avg_distance = np.mean(distances)
        
        # Store the average distance in the dictionary
        inter_class_distances[(class1, class2)] = avg_distance

# Now, inter_class_distances contains the average cosine distance between each pair of classes
for (class1, class2), avg_distance in inter_class_distances.items():
    print(f"Average Cosine Distance between Class {class1} and Class {class2}: {avg_distance}")


The following cell was generated by ChatGPT using the prompt:

> can the output from the above block be shown in a network graph?



In [0]:
import matplotlib.pyplot as plt
import numpy as np
import datetime  # Import the datetime module

# Create a list of class pairs and their corresponding average distances
class_pairs = list(inter_class_distances.keys())
avg_distances = list(inter_class_distances.values())

# Create a matrix of the average distances
n_classes = len(unique_classes)
dist_matrix = np.zeros((n_classes, n_classes))
for i in range(len(class_pairs)):
    class1, class2 = class_pairs[i]
    index1 = unique_classes.index(class1)
    index2 = unique_classes.index(class2)
    dist_matrix[index1, index2] = avg_distances[i]
    dist_matrix[index2, index1] = avg_distances[i]  # Set the symmetric value

# Plot the heatmap
plt.figure(figsize=(10, 8))
heatmap = plt.imshow(dist_matrix, cmap='coolwarm', interpolation='none')
plt.xticks(range(n_classes), unique_classes, rotation=90)
plt.yticks(range(n_classes), unique_classes)
plt.title("Average Cosine Distance between Classes")
plt.colorbar(label="Distance")

# Annotate the boxes with cosine similarity values rounded to two decimal places
for i in range(n_classes):
    for j in range(n_classes):
        plt.text(j, i, f'{dist_matrix[i, j]:.2f}', ha='center', va='center', color='white', fontsize=10)

# Get the current date and time
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the filename with the current date and time
filename = f"Avg_Cosine_Dist_bertbase_{current_datetime}.png"

# Save the figure
plt.savefig(filename)

plt.show()


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarities between embeddings
cosine_distances = 1 - cosine_similarity(embedd_array)


In [0]:
import matplotlib.pyplot as plt

# Create a heatmap of the cosine distances
plt.figure(figsize=(10, 8))
plt.imshow(cosine_distances, cmap='seismic', interpolation='none')
plt.colorbar(label='Cosine Distance')
plt.title('Cosine Distance Heatmap')
plt.xlabel('Requirements')
plt.ylabel('Requirements')
plt.show()


In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Specify the perplexity and learning rate values
perplexity_value = 30  # You can adjust this value
learning_rate_value = 30  # You can adjust this value
filename = f'tsne_visualization_bertbase_perplexity{perplexity_value}_learning_rate{learning_rate_value}.png'
# Initialize the t-SNE model with specified perplexity and learning rate
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

# Fit the t-SNE model to your data
embeddings_2d = tsne.fit_transform(embedd_array)

# Convert embeddings_2d array into a DataFrame with index values
df = pd.DataFrame(embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
df['document'] = documents

# Define colors and markers based on unique classes in the 'document' column
unique_classes = np.sort(df['document'].unique())  # Sort unique classes alphabetically
n_classes = len(unique_classes)
colors = sns.color_palette('Set1', n_colors=n_classes)  # Custom color palette
markers = 'o'  # Circle markers for all classes



# Create a scatter plot of the t-SNE embeddings for all classes
plt.figure(figsize=(10, 8))
# Set Seaborn style with gridlines
plt.figure(figsize=(10, 8))
sns.set(style='white')
handles = []
labels = []
for i, class_name in enumerate(unique_classes):
    df_class = df[df['document'] == class_name]
    handle = sns.scatterplot(data=df_class, x='Dimension 1', y='Dimension 2', hue='document', palette=[colors[i]],
                             marker=markers, edgecolor='k', s=75)
    handles.append(handle)
    labels.append(class_name)

# Create a legend for the classes
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, labels, title='Classes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel(filename, fontsize=12)
ax.set_ylabel('')

# Remove the title at the top
plt.title("")  # An empty string removes the title
#plt.title("t-SNE Visualization of Embeddings with Document Class Labels", fontsize=14)
#plt.xlabel("t-SNE Dimension 1", fontsize=12)
#plt.ylabel("t-SNE Dimension 2", fontsize=12)
plt.xticks([])
plt.yticks([])
plt.grid(False)  # Turn off gridlines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)




# Save the plot as a PNG image with perplexity and learning rate in the title
#filename = f'tsne_visualization_bertbase_perplexity{perplexity_value}_learning_rate{learning_rate_value}.png'
plt.savefig(filename, dpi=500, bbox_inches='tight')

plt.show()


In [0]:
import pandas as pd
import seaborn as sns

from sklearn.manifold import TSNE

# Specify the perplexity and learning rate values
perplexity_value = 50  # You can adjust this value
learning_rate_value = 30  # You can adjust this value

# Initialize the t-SNE model with specified perplexity and learning rate
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

# Fit the t-SNE model to your data
embeddings_2d = tsne.fit_transform(embedd_array)

# Create binary labels as a list (0 for 'requirement', 1 for 'standard')
labels = df.label.map({'standard': 1, 'requirement': 0}).tolist()

# Define colors and markers for the two classes
colors = sns.color_palette('Set1', n_colors=2)  # Custom color palette
markers = 'o'  # Circle markers for both classes

# Convert embeddings_2d array into a DataFrame with index values
df = pd.DataFrame(embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
df['label'] = labels

# Create a scatter plot of the t-SNE embeddings for both classes
plt.figure(figsize=(10, 8))
sns.set(style='whitegrid')  # Set Seaborn style with gridlines

for label in set(labels):
    df_label = df[df['label'] == label]
    sns.scatterplot(data=df_label, x='Dimension 1', y='Dimension 2', hue='label', palette=[colors[int(label)]], marker=markers, edgecolor='k', s=100)

# Create legends for the classes
class_labels = ['requirement', 'standard']  # Map the labels back to their original names
legend_handles = [plt.Line2D([0], [0], marker=markers, color='w', label=class_labels[int(label)], markersize=10,
                              markerfacecolor=colors[int(label)]) for label in set(labels)]
plt.legend(handles=legend_handles, title='Classes')

plt.title("t-SNE Visualization of Embeddings with Class Labels", fontsize=16)
plt.xlabel("t-SNE Dimension 1", fontsize=12)
plt.ylabel("t-SNE Dimension 2", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)  # Turn off gridlines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the plot as a PNG image
plt.savefig('tsne_visualization_basetraining.png', dpi=300, bbox_inches='tight')

plt.show()


In [0]:
import pandas as pd

# Create a DataFrame to store the points and their corresponding text values
df = pd.DataFrame({'Point': range(len(cluster_labels)), 'Cluster Label': cluster_labels})
df['Text'] = df['Point'].map(text_dict)  # Assuming text_dict is a dictionary mapping points to their text values
print(df)
Note: This assumes you have a dictionary called `text_dict` that maps each point to its corresponding text value. Adjust the code according to your specific setup.

In [0]:
# Import additional libraries
from sklearn.decomposition import PCA
import numpy as np

# Perform PCA on the embeddings
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_2d)

# Create a scatter plot of the PCA embeddings for both classes
plt.figure(figsize=(10, 8))
sns.set(style='whitegrid')  # Set Seaborn style with gridlines

for label in set(labels):
    df_label = df[df['label'] == label]
    sns.scatterplot(data=df_label, x='Dimension 1', y='Dimension 2', hue='label', palette=[colors[int(label)]], marker=markers, edgecolor='k', s=100)

plt.title("PCA Visualization of Embeddings with Class Labels", fontsize=16)
plt.xlabel("PCA Dimension 1", fontsize=12)
plt.ylabel("PCA Dimension 2", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)  # Turn off gridlines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the plot as a PNG image
plt.savefig('pca_visualization_basetraining.png', dpi=300, bbox_inches='tight')

plt.show()


In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Specify the perplexity and learning rate values
perplexity_value = 6  # You can adjust this value
learning_rate_value = 10  # You can adjust this value

# Initialize the t-SNE model with specified perplexity and learning rate
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

# Fit the t-SNE model to your data
embeddings_2d = tsne.fit_transform(embeddings)

# Create a DataFrame with the t-SNE embeddings and labels
df = pd.DataFrame(embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
df['label'] = labels
df['document_class'] = df['document']  # Create a new column for document classes

# Create a dictionary to map document classes to unique colors
unique_document_classes = df['document_class'].unique()
colors = sns.color_palette('husl', n_colors=len(unique_document_classes))
document_class_color_mapping = {doc_class: color for doc_class, color in zip(unique_document_classes, colors)}

# Assign a color to each data point based on its document class
df['color'] = df['document_class'].map(document_class_color_mapping)

# Create a scatter plot of the t-SNE embeddings for different document classes
plt.figure(figsize=(10, 8))
sns.set(style='whitegrid')  # Set Seaborn style with gridlines

for doc_class, color in document_class_color_mapping.items():
    df_class = df[df['document_class'] == doc_class]
    marker = 's' if doc_class == 'Triangle' else 'o'
    sns.scatterplot(data=df_class, x='Dimension 1', y='Dimension 2', hue='document_class', palette=[color], marker=marker, edgecolor='k', s=100)
    


# Create legends for the document classes
legend_handles = [plt.Line2D([0], [0], marker=marker, color='w', label=doc_class, markersize=10,
                              markerfacecolor=color) for doc_class, color in document_class_color_mapping.items()]
plt.legend(handles=legend_handles, title='Document Classes')

plt.title("t-SNE Visualization of Embeddings with Document Classes and IDs", fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(False)  # Turn off gridlines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the plot as a PNG image
plt.savefig('tsne_visualization_with_classes.png', dpi=300, bbox_inches='tight')

plt.show()


AttributeError: 'list' object has no attribute 'shape'